In [ ]:
from __future__ import print_function
import collections
import os
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np
import argparse

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Function to replace \n tags
def read_words(filename):
  with tf.io.gfile.GFile(filename, "rb") as f:
    return f.read().decode("utf-8").replace("\n", "").split()

In [ ]:
# Function to create a dictionary of words and a corresponding ID for each word
def build_vocab(filename):
  data = read_words(filename)

  counter = collections.Counter(data)
  count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

  words, _ = list(zip(*count_pairs))
  word_to_id = dict(zip(words, range(len(words))))

  return word_to_id

In [ ]:
#Function to convert each word with specific ID
def file_to_word_ids(filename, word_to_id):
  data = read_words(filename)
  return [word_to_id[word] for word in data if word in word_to_id]

In [ ]:
def load_data():

  train_path = "/content/gdrive/MyDrive/ptbdataset/ptb.train.txt"
  valid_path = "/content/gdrive/MyDrive/ptbdataset/ptb.valid.txt"
  test_path = "/content/gdrive/MyDrive/ptbdataset/ptb.test.txt"

  #Build vocabulary then convert text to int list
  word_to_id = build_vocab(train_path)
  train_data = file_to_word_ids(train_path, word_to_id)
  valid_data = file_to_word_ids(valid_path, word_to_id)
  test_data = file_to_word_ids(test_path, word_to_id)
  vocabulary = len(word_to_id)
  reversed_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))

  print(train_data[:5])
  print(word_to_id)
  print(vocabulary)
  print(" ".join([reversed_dictionary[x] for x in train_data[:10]]))
  return train_data, valid_data, test_data, vocabulary, reversed_dictionary

In [ ]:
train_data, valid_data, test_data, vocabulary, reversed_dictionary = load_data()

[9969, 9970, 9971, 9973, 9974]
{'the': 0, '<unk>': 1, 'N': 2, 'of': 3, 'to': 4, 'a': 5, 'in': 6, 'and': 7, "'s": 8, 'that': 9, 'for': 10, '$': 11, 'is': 12, 'it': 13, 'said': 14, 'on': 15, 'by': 16, 'at': 17, 'as': 18, 'from': 19, 'million': 20, 'with': 21, 'mr.': 22, 'was': 23, 'be': 24, 'are': 25, 'its': 26, 'he': 27, 'but': 28, 'has': 29, 'an': 30, "n't": 31, 'will': 32, 'have': 33, 'new': 34, 'or': 35, 'company': 36, 'they': 37, 'this': 38, 'year': 39, 'which': 40, 'would': 41, 'about': 42, 'says': 43, 'more': 44, 'were': 45, 'market': 46, 'billion': 47, 'his': 48, 'had': 49, 'their': 50, 'up': 51, 'u.s.': 52, 'one': 53, 'than': 54, 'who': 55, 'some': 56, 'been': 57, 'also': 58, 'stock': 59, 'other': 60, 'share': 61, 'not': 62, 'we': 63, 'corp.': 64, 'if': 65, 'when': 66, 'i': 67, 'last': 68, 'president': 69, 'shares': 70, 'years': 71, 'all': 72, 'first': 73, 'two': 74, 'because': 75, 'trading': 76, 'after': 77, 'could': 78, 'inc.': 79, 'sales': 80, '&': 81, 'there': 82, 'out': 83,

In [ ]:
class KerasBatchGenerator(object):

  def __init__(self, data, num_steps, batch_size, vocabulary, skip_step=5):
    self.data = data
    self.num_steps = num_steps
    self.batch_size = batch_size
    self.vocabulary = vocabulary
    # this will track the progress of the batches sequentially through the data set - 
    # once the data reaches the end of the data set it will reset back to zero
    self.current_idx = 0
    # skip_step is the number of words which will be skipped before the next
    # batch is skimmed from the data set
    self.skip_step = skip_step

  def generate(self):
        x = np.zeros((self.batch_size, self.num_steps))
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary))
        while True:
            for i in range(self.batch_size):
                if self.current_idx + self.num_steps >= len(self.data):
                    # reset the index back to the start of the data set
                    self.current_idx = 0
                x[i, :] = self.data[self.current_idx:self.current_idx + self.num_steps]
                temp_y = self.data[self.current_idx + 1:self.current_idx + self.num_steps + 1]
                # convert all of temp_y into a one hot representation
                y[i, :, :] = to_categorical(temp_y, num_classes=self.vocabulary)
                self.current_idx += self.skip_step
            yield x, y

In [ ]:
num_steps = 32
batch_size = 64

train_data_generator = KerasBatchGenerator(train_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)
valid_data_generator = KerasBatchGenerator(valid_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)

hidden_size = 2048 
use_dropout = True

model = Sequential()
model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(Dropout(0.4))
model.add(TimeDistributed(Dense(vocabulary)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='categorical_accuracy')

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 32, 2048)          20477952  
                                                                 
 lstm_12 (LSTM)              (None, 32, 2048)          33562624  
                                                                 
 dropout_12 (Dropout)        (None, 32, 2048)          0         
                                                                 
 lstm_13 (LSTM)              (None, 32, 2048)          33562624  
                                                                 
 dropout_13 (Dropout)        (None, 32, 2048)          0         
                                                                 
 time_distributed_6 (TimeDis  (None, 32, 9999)         20487951  
 tributed)                                                       
                                                      

In [ ]:
num_epochs = 50

model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,
                    validation_data=valid_data_generator.generate(),
                    validation_steps=len(valid_data)//(batch_size*num_steps))

Epoch 1/50


<ipython-input-29-47f5d13681f0>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,


433/433 [==============================] - 180s 409ms/step - loss: 6.4411 - categorical_accuracy: 0.1022 - val_loss: 5.9314 - val_categorical_accuracy: 0.1419
Epoch 2/50
433/433 [==============================] - 162s 374ms/step - loss: 5.7699 - categorical_accuracy: 0.1541 - val_loss: 5.5889 - val_categorical_accuracy: 0.1698
Epoch 3/50
433/433 [==============================] - 162s 374ms/step - loss: 5.4382 - categorical_accuracy: 0.1771 - val_loss: 5.4178 - val_categorical_accuracy: 0.1868
Epoch 4/50
433/433 [==============================] - 162s 374ms/step - loss: 5.1948 - categorical_accuracy: 0.1919 - val_loss: 5.3238 - val_categorical_accuracy: 0.1957
Epoch 5/50
433/433 [==============================] - 162s 374ms/step - loss: 4.9775 - categorical_accuracy: 0.2039 - val_loss: 5.2499 - val_categorical_accuracy: 0.2038
Epoch 6/50
433/433 [==============================] - 162s 374ms/step - loss: 4.7896 - categorical_accuracy: 0.2138 - val_loss: 5.2086 - val_categorical_accuracy